In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## ExcelからDataを読み込む
- Weather: sunny = 1, rainy = 0
- Car: working = 1, broken = 0
- Class: go-out = 1, stay-home = 0

In [2]:
# Excel内の画像などは読み込めないというwarningが出るが, dataは読み込めているため無視する. 
data = pd.read_excel("09-NaiveBayes.xlsx", usecols=[0, 1, 2], header=17)[:10].astype(np.float64)
data

/Users/tobiuo/learning/machine-learning/venv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Weather,Car,Class
0,1.0,1.0,1.0
1,0.0,0.0,1.0
2,1.0,1.0,1.0
3,1.0,1.0,1.0
4,1.0,1.0,1.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,1.0,1.0,0.0
8,1.0,0.0,0.0
9,0.0,0.0,0.0


## Classの種類を記述する

In [3]:
weather = ['sunny', 'rainy']
car = ['working', 'broken']
outs = ['go_out', 'stay_home'] # classという変数名は使えないのでoutsで代用

statuses = [weather, car, outs]

## 各Classの基本確率を求める

In [4]:
count_out = [len(data.query('Class == '+str(out))) for out in range(len(outs))]

## 要素ごとに集合を作る

In [5]:
aggregations = copy.deepcopy(statuses)
for status in range(len(statuses)):
    for state in range(len(statuses[status])):
        expr = str(data.columns[status]) +'=='+ str(state)
        aggregations[status][state] = set(data.query(expr).index)

## 各事象の確率を求める

In [6]:
probs = []
for status in range(len(aggregations) - 1):
    probs.append([])
    for state in range(len(aggregations[status])):
        probs[status].append([])
        for out in range(len(aggregations[-1])):
            event = aggregations[status][state]&aggregations[-1][out]
            prob = len(event) / count_out[out]
            probs[status][state].append(prob)

## 尤度を計算し, Classを予測する

In [7]:
predict = []
for i, d in data.iterrows():
    likelihoods = []
    for out in range(len(aggregations[-1])):
        likelihood = count_out[out] / len(data)
        for status in range(len(aggregations) - 1):
            likelihood *= probs[status][int(d[status])][out]
        likelihoods.append(np.round(likelihood, 2))
    predict.append(likelihoods.index(max(likelihoods)))
        
predict

[1, 0, 1, 1, 1, 0, 0, 1, 0, 0]

## 精度を確かめる

In [8]:
accuracy = (predict == data.Class).sum() / len(data)
print(str(accuracy * 100) +'%')

80.0%
